In [4]:
pip install pandas sqlalchemy requests && pip install psycopg2-binary --quiet

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.1 MB/s eta 0:00:00ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
from sqlalchemy import create_engine, select, func, column, table, asc, desc
import logging
import os
import requests
import ast
from urllib.parse import urlparse
import sys
import math
import ipywidgets as widgets
from IPython.display import display, clear_output

In [6]:
df_anotado = pd.read_csv('/home/jovyan/vlm_benchmark-feature-200-benchmark/data/CSVs/300_imoveis_anotados.csv')
print("300 Imóveis anotados")

df_anotado

300 Imóveis anotados


,url,id_imovel,id_anuncio,tipo_imovel,titulo,descricao,acabamento_externo,acabamento_interno,cobertura,esquadrias,estrutura,forro,instalacao_eletrica,instalacao_sanitaria,piso,revestimento_externo,revestimento_interno
0,www.vivareal.com.br/imovel/casa-3-quartos-anha...,00023474-dffe-37cb-9813-d32e8c980294,2811629951,HOME,Casa à venda com 3 suítes Parque Anhanguera.,"Casa com 3 suítes plenas, ampla sala com dois ...",pintura lavavel,pintura lavavel,telha de barro,aluminio,concreto,especial,embutida,mais de uma,especial/porcelanato,massa,massa
1,www.vivareal.com.br/imovel/sobrado-3-quartos-s...,0010dc46-f9ea-387f-b2bf-c5f076ffae89,2816312120,HOME,Sobrado reformado 3 suítes no Setor Faiçalville,"Sobrado com 3 suítes no Setor Faiçalville, sen...",pintura lavavel,pintura lavavel,laje,aluminio,concreto,especial,embutida,mais de uma,especial/porcelanato,massa,massa
2,www.vivareal.com.br/imovel/casa-3-quartos-seto...,00121046-ee74-3413-b5e2-63514b185031,2810514952,HOME,"Casa 3 suites prox Hipodromo Lagoinha, nova, p...","Casa 3 suites, sala ampla dois ambientes com c...",pintura lavavel,pintura lavavel,laje,aluminio,concreto,especial,embutida,mais de uma,especial/porcelanato,massa,massa
3,www.vivareal.com.br/imovel/casa-3-quartos-tres...,0012b055-c750-30ce-a92f-1cb9835fbf40,2816363457,HOME,Casa com 3 suítes no Setor Três Marias. codigo...,Casa Alto Padrão<br>Setor Três Marias - Goiâni...,especial,pintura lavavel,laje,aluminio,concreto,especial,embutida,mais de uma,especial/porcelanato,especial,massa
4,www.vivareal.com.br/imovel/casa-3-quartos-seto...,0022aeba-4b7d-38c9-a370-61aaa0ed5c0b,2817817445,HOME,Casa Térrea St Faicalville 228 metros quadrado...,Excelente casa térrea pronta pra mora alto pad...,pintura lavavel,pintura lavavel,laje,aluminio,concreto,especial,embutida,mais de uma,especial/porcelanato,massa,massa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,www.vivareal.com.br/imovel/ponto-comercial-6-q...,27e8a901-5bbd-3934-9317-8c2c61f0ec79,2780848836,BUSINESS,PRÉDIO COMERCIAL NO CENTRO DE GOIÂNIA,VENDO SALAS COMERCIAIS COM ESTRUTURA PARA CLIN...,pintura lavavel,pintura lavavel,laje,aluminio,alvenaria,laje,embutida,mais de uma,especial/porcelanato,massa,material ceramico
291,www.vivareal.com.br/imovel/ponto-comercial-set...,28a0f908-06be-3af6-aa73-4a7f5bf6de89,2740064723,BUSINESS,"Prédio Comercial à Venda, 730m2 Centro de Goiânia",Casa Comercial no Centro de Goiânia Dois Lote...,pintura lavavel,pintura lavavel,telha de barro,aluminio,alvenaria,laje,embutida,interna,ceramica,massa,massa
292,www.vivareal.com.br/imovel/ponto-comercial-set...,28eea1d7-cb89-3743-9c32-983fc79a4c23,2821376718,BUSINESS,"Açougue, Casa de Carne e Empório no St Bueno á...",Vendo uma casa de carne no St Bueno em Goiânia...,pintura lavavel,pintura lavavel,laje,especial,alvenaria,laje,embutida,interna,cimento,massa,material ceramico
293,www.vivareal.com.br/imovel/ponto-comercial-set...,291c70aa-5dd2-309f-a0b7-410591423f09,2790148116,BUSINESS,Localizada em um dos pontos mais estratégicos...,Localizada em um dos pontos mais estratégicos...,pintura lavavel,pintura lavavel,laje,especial,concreto,laje,embutida,interna,taco,massa,massa


In [7]:
class FormatadorCustomizado(logging.Formatter):
    cinza = "\x1b[38;20m"
    verde = "\x1b[32m"
    amarelo = "\x1b[33m"
    vermelho = "\x1b[31m"
    vermelho_negrito = "\x1b[31;1m"
    resetar = "\x1b[0m"
    
    formato_log = "%(asctime)s - %(levelname)-8s - %(message)s (%(filename)s:%(lineno)d)"

    FORMATOS = {
        logging.DEBUG: cinza + formato_log + resetar,
        logging.INFO: verde + formato_log + resetar,
        logging.WARNING: amarelo + formato_log + resetar,
        logging.ERROR: vermelho + formato_log + resetar,
        logging.CRITICAL: vermelho_negrito + formato_log + resetar
    }

    def format(self, registro):
        formato_log = self.FORMATOS.get(registro.levelno)
        formatador = logging.Formatter(formato_log)
        return formatador.format(registro)

logger = logging.getLogger("vlm-test-automation")
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler()
handler.setLevel(logging.DEBUG)
handler.setFormatter(FormatadorCustomizado())
if not logger.handlers:
    logger.addHandler(handler)

In [8]:
# --- Variáveis de Conexão e Configuração ---
DB_USUARIO = 'crawler_user'
DB_SENHA = 'cZ6-12"7u1f&'
DB_HOST = 'postgres.postgresql-server.svc.cluster.local'
DB_PORTA = '5432'
DB_NOME = 'web_crawler'

QUANTIDADE_IMOVEIS_ANOTADOS = len(df_anotado)
QUANTIDADE_IMOVEIS_TOTAL = 500

TIPOS_IMOVEIS_PARA_FILTRAR = ['HOME', 'APARTMENT', 'CONDOMINIUM', 'BUSINESS']

QUANTIDADE_IMOVEIS_POR_TIPO = math.ceil((QUANTIDADE_IMOVEIS_TOTAL - QUANTIDADE_IMOVEIS_ANOTADOS)/len(TIPOS_IMOVEIS_PARA_FILTRAR))

COLUNAS_PARA_SELECIONAR = [
    'id_imovel',
    'id_anuncio',
    'url',
    'imagens',
    'titulo',
    'descricao',
    'tipo_imovel',
    'data_atualizacao',
    'data_criacao'
]

FILTRAR_POR_ID_IMOVEL = True
COLUNA_ORDENACAO = 'data_atualizacao'
TIPO_ORDENACAO = 'DESC'
OFFSET = 0
ALEATORIO = True

In [9]:
def verificar_dependencias():
    """Verifica se todas as dependências necessárias estão instaladas"""
    try:
        import psycopg2
        logger.info("psycopg2 encontrado.")
        return True
    except ImportError:
        logger.error("psycopg2 não encontrado. Instalando...")
        try:
            import subprocess
            import sys
            subprocess.check_call([sys.executable, "-m", "pip", "install", "psycopg2-binary"])
            logger.info("psycopg2-binary instalado com sucesso.")
            return True
        except Exception as e:
            logger.critical(f"Erro ao instalar psycopg2-binary: {e}")
            return False

def criar_conexao_bd():
    """Cria e testa a conexão com o banco de dados"""
    try:
        motor_bd = create_engine(
            f"postgresql://{DB_USUARIO}:{DB_SENHA}@{DB_HOST}:{DB_PORTA}/{DB_NOME}",
            pool_pre_ping=True,  # Verifica se a conexão está ativa
            pool_recycle=3600    # Reconecta a cada hora
        )
        
        # Testa a conexão
        with motor_bd.connect() as conn:
            conn.execute(select(func.current_timestamp()))
        
        logger.info("Conexão com o banco de dados estabelecida com sucesso.")
        return motor_bd
    except Exception as e:
        logger.critical(f"Erro ao conectar ao banco de dados: {e}")
        return None

def baixar_imagens(df, pasta_destino="imoveis_fotos", max_imagens_por_imovel=5):
    """Baixa imagens com controle de memória"""
    if 'imagens' not in df.columns:
        logger.warning("Coluna 'imagens' não encontrada no DataFrame. O download de fotos será ignorado.")
        return

    os.makedirs(pasta_destino, exist_ok=True)
    logger.info(f"Iniciando o download de imagens na pasta: {pasta_destino}")

    total_imoveis = len(df)
    for idx, (_, linha) in enumerate(df.iterrows()):
        try:
            tipo_imovel = linha['tipo_imovel']
            id_anuncio = linha['id_anuncio']
            lista_imagens_str = linha['imagens']

            if pd.isna(lista_imagens_str) or not lista_imagens_str:
                logger.debug(f"Anúncio {id_anuncio} não tem imagens. Pulando.")
                continue

            try:
                lista_urls = [url.strip() for url in lista_imagens_str.strip('{}').split(',')]
                lista_urls = [url for url in lista_urls if url][:max_imagens_por_imovel]  # Limita número de imagens
            except Exception as e:
                logger.error(f"Erro ao processar a string de imagens do anúncio {id_anuncio}: {e}")
                continue

            pasta_imovel = os.path.join(pasta_destino, str(tipo_imovel), str(id_anuncio))
            os.makedirs(pasta_imovel, exist_ok=True)

            for i, url_imagem in enumerate(lista_urls):
                try:
                    resposta = requests.get(url_imagem, timeout=10, stream=True)
                    resposta.raise_for_status()
                    
                    nome_arquivo = os.path.basename(urlparse(url_imagem).path)
                    if not nome_arquivo:
                        nome_arquivo = f"imagem_{i+1}.jpg"
                    
                    caminho_arquivo = os.path.join(pasta_imovel, nome_arquivo)
                    with open(caminho_arquivo, 'wb') as f:
                        for chunk in resposta.iter_content(chunk_size=8192):
                            f.write(chunk)
                    
                    logger.debug(f"Imagem {i+1} de {id_anuncio} baixada com sucesso.")
                except requests.exceptions.RequestException as e:
                    logger.error(f"Erro ao baixar imagem '{url_imagem}' do anúncio {id_anuncio}: {e}")
                except Exception as e:
                    logger.error(f"Erro inesperado ao processar imagem {url_imagem}: {e}")
        
        except Exception as e:
            logger.error(f"Erro ao processar imóvel {idx + 1}/{total_imoveis}: {e}")
        
        # Log de progresso a cada 10 imóveis
        if (idx + 1) % 10 == 0:
            logger.info(f"Processados {idx + 1}/{total_imoveis} imóveis")

    logger.info("Processo de download de imagens concluído.")

In [13]:
def main():
    """Função principal com tratamento de erros melhorado"""
    
    # Verificar dependências
    if not verificar_dependencias():
        logger.critical("Dependências não atendidas. Encerrando execução.")
        return False
    
    # Criar conexão
    motor_bd = criar_conexao_bd()
    if motor_bd is None:
        logger.critical("Não foi possível estabelecer conexão com o banco. Encerrando execução.")
        return False
    
    try:
        # Configurar tabela
        tabela_zap = table(
            'vivareal',
            *(column(col) for col in COLUNAS_PARA_SELECIONAR),
            schema='prata'
        )

        # Configurar ordenação
        clausula_ordenacao = None
        if ALEATORIO:
            clausula_ordenacao = func.random()
            logger.info("Modo de seleção aleatória ativado.")
        elif TIPO_ORDENACAO.upper() == 'ASC':
            clausula_ordenacao = asc(tabela_zap.c[COLUNA_ORDENACAO])
            logger.info(f"Ordenando por '{COLUNA_ORDENACAO}' em ordem crescente.")
        else:
            clausula_ordenacao = desc(tabela_zap.c[COLUNA_ORDENACAO])
            logger.info(f"Ordenando por '{COLUNA_ORDENACAO}' em ordem decrescente.")

        # Montar query
        colunas_query_principal = [tabela_zap.c[col] for col in COLUNAS_PARA_SELECIONAR]

        if FILTRAR_POR_ID_IMOVEL:
            logger.info("Filtragem por primeira ocorrência de id_imovel ativada.")
            cte_query = select(
                tabela_zap,
                func.row_number().over(
                    partition_by=tabela_zap.c.id_imovel,
                    order_by=clausula_ordenacao
                ).label('rn')
            ).cte('primeira_ocorrencia')

            colunas_query_principal = [cte_query.c[col] for col in COLUNAS_PARA_SELECIONAR]
            
            query = select(
                *colunas_query_principal
            ).where(
                cte_query.c.rn == 1
            ).where(
                cte_query.c.tipo_imovel.in_(TIPOS_IMOVEIS_PARA_FILTRAR)
            )
        else:
            logger.warning("Filtragem por primeira ocorrência de id_imovel desativada.")
            query = select(
                *colunas_query_principal
            ).where(
                tabela_zap.c.tipo_imovel.in_(TIPOS_IMOVEIS_PARA_FILTRAR)
            ).order_by(
                clausula_ordenacao
            )

        # Executar query
        logger.info("Executando consulta SQL...")
        df_imoveis = pd.read_sql(query, motor_bd)
        logger.info(f"Consulta SQL executada. {len(df_imoveis)} registros encontrados.")
        
        if df_imoveis.empty:
            logger.warning("Nenhum registro encontrado. Verifique os filtros.")
            return False

        # Remover colunas desnecessárias
        try:
            colunas_remover = [
                col for col in ['data_atualizacao', 'data_criacao'] 
                if col in df_imoveis.columns and col != COLUNA_ORDENACAO
            ]
            if colunas_remover:
                df_imoveis.drop(columns=colunas_remover, inplace=True)
                logger.info(f"Colunas temporárias de ordenação removidas: {colunas_remover}.")
        except KeyError as e:
            logger.warning(f"Não foi possível remover coluna de ordenação: {e}")

        # Remover imóveis iguais aos já anotados
        df_imoveis = df_imoveis[~df_imoveis['id_imovel'].isin(df_anotado['id_imovel'])]

        # Filtrar por tipo e quantidade
        imoveis_filtrados = []
        for tipo in TIPOS_IMOVEIS_PARA_FILTRAR:
            df_tipo = df_imoveis[df_imoveis['tipo_imovel'] == tipo]
            df_tipo_limitado = df_tipo.iloc[OFFSET : OFFSET + QUANTIDADE_IMOVEIS_POR_TIPO]
            imoveis_filtrados.append(df_tipo_limitado)
            logger.info(f"Selecionados {len(df_tipo_limitado)} registros do tipo '{tipo}'.")

        df_final = pd.concat(imoveis_filtrados, ignore_index=True)
        logger.info(f"DataFrame final criado com {len(df_final)} registros.")

        # Salvar CSV antes do download de imagens
        nome_arquivo_saida = 'imoveis_personalizados.csv'
        df_final.to_csv(nome_arquivo_saida, index=False)
        logger.info(f"DataFrame exportado com sucesso para o arquivo: {nome_arquivo_saida}")

        # Função que será chamada quando o botão for clicado
        def on_button_click(b):
            clear_output()  # limpa os botões após o clique
            if b.description == "Sim":
                print("Fazendo download das imagens...")
                baixar_imagens(df_final)
            else:
                logger.info("Download de imagens pulado.")

        # Criação dos botões
        botao_sim = widgets.Button(description="Sim", button_style='success')  # verde
        botao_nao = widgets.Button(description="Não", button_style='danger')   # vermelho

        # Ligando os botões à função
        botao_sim.on_click(on_button_click)
        botao_nao.on_click(on_button_click)

        # Mostrando na tela
        print("Deseja fazer o download das imagens?")
        display(widgets.HBox([botao_sim, botao_nao]))

        '''# Download de imagens (opcional)
        resposta = input("Deseja fazer o download das imagens? (s/n): ").lower().strip()
        if resposta in ['s', 'sim', 'y', 'yes']:
            baixar_imagens(df_final)
        else:
            logger.info("Download de imagens pulado.")'''

        logger.info("Execução concluída com sucesso!")
        return True
        
    except Exception as e:
        logger.error(f"Erro durante a execução: {e}")
        return False
    finally:
        # Fechar conexão
        if motor_bd:
            motor_bd.dispose()
            logger.info("Conexão com banco de dados fechada.")

        

In [14]:
# Executar apenas se for o script principal
if __name__ == "__main__":
    sucesso = main()
    if not sucesso:
        sys.exit(1)

2025-10-17 04:04:27,595 - INFO     - Iniciando o download de imagens na pasta: imoveis_fotos (2466637600.py:45)


Fazendo download das imagens...


2025-10-17 04:04:28,306 - DEBUG    - Imagem 1 de 2816332318 baixada com sucesso. (2466637600.py:82)
2025-10-17 04:04:28,421 - DEBUG    - Imagem 2 de 2816332318 baixada com sucesso. (2466637600.py:82)
2025-10-17 04:04:28,517 - DEBUG    - Imagem 3 de 2816332318 baixada com sucesso. (2466637600.py:82)
2025-10-17 04:04:28,666 - DEBUG    - Imagem 1 de 2775548641 baixada com sucesso. (2466637600.py:82)
2025-10-17 04:04:28,960 - DEBUG    - Imagem 2 de 2775548641 baixada com sucesso. (2466637600.py:82)
2025-10-17 04:04:30,367 - DEBUG    - Imagem 3 de 2775548641 baixada com sucesso. (2466637600.py:82)
2025-10-17 04:04:30,625 - DEBUG    - Imagem 4 de 2775548641 baixada com sucesso. (2466637600.py:82)
2025-10-17 04:04:30,726 - DEBUG    - Imagem 5 de 2775548641 baixada com sucesso. (2466637600.py:82)
2025-10-17 04:04:31,302 - DEBUG    - Imagem 1 de 2820027013 baixada com sucesso. (2466637600.py:82)
2025-10-17 04:04:31,637 - DEBUG    - Imagem 2 de 2820027013 baixada com sucesso. (2466637600.py:82)
